In [966]:
import pandas as pd

In [967]:
train=pd.read_csv('home-data-for-ml-course\\train.csv')
test=pd.read_csv('home-data-for-ml-course\\test.csv')

train.columns = train.columns.str.strip()
test.columns = test.columns.str.strip()

Y_train=train['SalePrice']
train.drop(columns=['SalePrice'],inplace=True)


In [968]:
data=pd.concat([train,test],axis=0,ignore_index=True)
Ids= data['Id']
data.drop(columns=['Id'],inplace=True)
print(data.shape)

(2919, 79)


In [969]:
data[["LotFrontage", "MasVnrArea", "GarageYrBlt"]] = data[["LotFrontage", "MasVnrArea", "GarageYrBlt"]].apply(pd.to_numeric, errors="coerce")

data_categorical_cols = [feature for feature in data.columns if data[feature].dtype == "object"] 

data_nominal_cols = ["MSZoning", "Street", "Alley", "LandContour", "LotConfig", "Neighborhood", "Condition1", "Condition2", 
                    "BldgType", "HouseStyle", "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating",
                    "CentralAir", 'Electrical',"GarageType", "MiscFeature", "SaleType", "SaleCondition"]

data_ordinal_cols = [ 'LotShape','Utilities','LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1',
                    'BsmtFinType2','HeatingQC','KitchenQual','Functional','FireplaceQu','GarageFinish','GarageQual',
                    'GarageCond','PavedDrive','PoolQC','Fence']

data_numerical_cols = [feature for feature in data.columns if feature not in data_categorical_cols]
data_numerical_cols_without_SalePrice = [feature for feature in data.columns if feature not in data_categorical_cols and feature!='SalePrice']

data_discrete_numerical_cols = [ 'OverallQual','OverallCond','BsmtFullBath','BsmtHalfBath',
  'FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr','TotRmsAbvGrd','Fireplaces','GarageCars','MoSold', "MSSubClass"] 

data_continuous_numerical_cols = ['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
                                  'BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','GarageYrBlt',
                                  'GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea',
                                  'MiscVal','YrSold','SalePrice']
for col in data_numerical_cols_without_SalePrice:
    train[col] = pd.to_numeric(train[col], errors='coerce')  
    test[col] = pd.to_numeric(test[col], errors='coerce')
  # Remove spaces from all categorical features
for col in data_categorical_cols:
    train[col] = train[col].str.strip().replace(" ", "")
    test[col] = test[col].str.strip().replace(" ", "")

  
def update_cols():
    global data_nominal_cols, data_ordinal_cols,data_continuous_numerical_cols,data_discrete_numerical_cols,data_categorical_cols,data_numerical_cols
    data_categorical_cols=[col for col in data.columns if col in data_categorical_cols]
    data_numerical_cols=[col for col in data.columns if col in data_numerical_cols]
    data_nominal_cols=[col for col in data.columns if col in data_nominal_cols]
    data_ordinal_cols=[col for col in data.columns if col in data_ordinal_cols]
    data_continuous_numerical_cols=[col for col in data.columns if col in data_continuous_numerical_cols]
    data_discrete_numerical_cols=[col for col in data.columns if col in data_discrete_numerical_cols]


Missing Values

In [970]:
data.isnull().sum().sort_values(ascending=False).head(10)

PoolQC          2909
MiscFeature     2814
Alley           2721
Fence           2348
MasVnrType      1766
FireplaceQu     1420
LotFrontage      486
GarageYrBlt      159
GarageFinish     159
GarageQual       159
dtype: int64

In [971]:
# dropping columns with too many null values
data.drop(columns=['PoolQC','MiscFeature','Alley','Fence','MasVnrType','FireplaceQu'],inplace=True)
update_cols()
print(data.shape)

(2919, 73)


In [972]:
common_value_threshold=95
def most_commun(threashold=80):
    dict=[]
    for col in data.columns:
        most_commun_value=data[col].mode()[0]
        most_commun_value_percentage=round((data[col]==most_commun_value).mean(),4)
        dict.append({'Feature': col,'Most Common Value': most_commun_value,'Percentage': most_commun_value_percentage*100})    
    dict= pd.DataFrame(dict)
    return dict[dict['Percentage']>threashold].sort_values(by='Percentage',ascending=False)


cols_with_too_many_same_values=list(most_commun(common_value_threshold)['Feature'])
print(cols_with_too_many_same_values)
most_commun(90)


['Utilities', 'Street', 'PoolArea', 'Condition2', '3SsnPorch', 'LowQualFinSF', 'RoofMatl', 'Heating', 'MiscVal', 'KitchenAbvGr', 'LandSlope']


,Feature,Most Common Value,Percentage
7,Utilities,AllPub,99.90
4,Street,Pave,99.59
67,PoolArea,0,99.55
12,Condition2,Norm,98.97
65,3SsnPorch,0,98.73
42,LowQualFinSF,0,98.63
20,RoofMatl,CompShg,98.53
36,Heating,GasA,98.46
68,MiscVal,0,96.47
49,KitchenAbvGr,1,95.41


In [973]:
# dropping features that have mostly one value
data.drop(columns=cols_with_too_many_same_values,inplace=True)
update_cols()
print(data.shape)



(2919, 62)


In [974]:
numerical_cols = list(data.select_dtypes(include=['number']).columns)
categorical_cols= list(data.select_dtypes(include=['object']).columns)
print('Numerical: ',len(numerical_cols),'Categorical: ',len(categorical_cols))

data.shape

Numerical:  31 Categorical:  31


(2919, 62)

In [975]:
from sklearn.impute import SimpleImputer
print('data before imputing: ',data.isnull().sum().sum())

update_cols()

mode_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')

data[categorical_cols]=mode_imputer.fit_transform(data[categorical_cols])
data[data_discrete_numerical_cols]=mode_imputer.fit_transform(data[data_discrete_numerical_cols])
data[data_continuous_numerical_cols]=mean_imputer.fit_transform(data[data_continuous_numerical_cols])

print('data after imputing: ',data.isnull().sum().sum())



data before imputing:  1727
data after imputing:  0


Encoding 

In [976]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

encoder = LabelEncoder()

for col in data_categorical_cols:
    data[col] = encoder.fit_transform(data[col])

data.shape



(2919, 62)

Correlation

In [977]:
import numpy as np
pair_corr_threshold=0.75

corr_matrix = data[numerical_cols].corr()

upper_triangle = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
high_corr_pairs = [
    (corr_matrix.index[i], corr_matrix.columns[j], corr_matrix.iat[i, j])
    for i in range(corr_matrix.shape[0]) 
    for j in range(corr_matrix.shape[1]) 
    if upper_triangle[i, j] and abs(corr_matrix.iat[i, j]) > pair_corr_threshold  # Threshold for high correlation
]

for f1, f2, correlation in high_corr_pairs:
    print(f"({f1}, {f2}, {correlation:.4f})")


cols_to_drop=['GarageYrBlt','TotalBsmtSF','GarageCars','TotRmsAbvGrd']


(YearBuilt, GarageYrBlt, 0.7900)
(TotalBsmtSF, 1stFlrSF, 0.8016)
(GrLivArea, TotRmsAbvGrd, 0.8084)
(GarageCars, GarageArea, 0.8897)


In [978]:
corr_threshold=0.012
data['SalePrice']=Y_train
correlation_with_target = data.loc[:1460,:].corrwith(data.loc[:1460, 'SalePrice'])

correlation_with_target_df = correlation_with_target.abs().sort_values(ascending=False).reset_index()
correlation_with_target_df.columns = ['Feature', 'Correlation']

data.drop(columns='SalePrice',inplace=True)
low_corr_features= correlation_with_target_df[correlation_with_target_df['Correlation'] <= corr_threshold].sort_values(by='Correlation',ascending=False)
cols_to_drop.extend(list(low_corr_features['Feature']))
print((len(cols_to_drop)-4),'Features with less than ', corr_threshold, 'with SalePrice')

low_corr_features

1 Features with less than  0.012 with SalePrice


,Feature,Correlation
62,BsmtFinSF2,0.011378


In [979]:
print('Columns to drop: ', cols_to_drop)
print("before: ",data.shape)
data.drop(columns=cols_to_drop,inplace=True)
print("after: ",data.shape)
update_cols()

Columns to drop:  ['GarageYrBlt', 'TotalBsmtSF', 'GarageCars', 'TotRmsAbvGrd', 'BsmtFinSF2']
before:  (2919, 62)
after:  (2919, 57)


Feature Engineering

In [980]:
print(data.shape)

data['TotalBath'] = (data['FullBath'] + (0.5 * data['HalfBath']) +
                     data['BsmtFullBath'] + (0.5 * data['BsmtHalfBath']))
data['TotalPorchSF'] = (data['OpenPorchSF'] + data['EnclosedPorch'] +
                         data['ScreenPorch'] + data['WoodDeckSF'])
data['HouseAge'] = data['YrSold'] - data['YearBuilt']
data['RemodAge'] = data['YrSold'] - data['YearRemodAdd']
data['HasGarage'] = data['GarageArea'].apply(lambda x: 1 if x > 0 else 0)

data['Quality_LivingArea'] = data['OverallQual'] * data['GrLivArea']
data['Baths_per_Bedroom'] = data['TotalBath'] / (data['BedroomAbvGr'] + 1)

data.drop(columns=['GrLivArea', 'FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath',  
                   'EnclosedPorch', 'WoodDeckSF',  
                    'YearRemodAdd' ], inplace=True)

print(data.shape)
update_cols()

(2919, 57)
(2919, 56)


In [ ]:
data['Id']=Ids
data['SalePrice']=Y_train


Q1 = data.iloc[0:1460,:][data_numerical_cols].quantile(0.25)
Q3 = data.iloc[0:1460,:][data_numerical_cols].quantile(0.75)
IQR = Q3 - Q1

k = 5
outlier_mask = ((data.iloc[0:1460,:][data_numerical_cols] < (Q1 - k * IQR)) | 
                (data.iloc[0:1460,:][data_numerical_cols] > (Q3 + k * IQR))).any(axis=1)
outlier_ids = data.loc[data.index[:1460][outlier_mask], 'Id'].tolist()
outliers_removed=len(outlier_ids)
print(outliers_removed, "outliers found")
print(outlier_ids)

print('data before removing: ',data.shape)
data.drop(data[data['Id'].isin(outlier_ids)].index, inplace=True)
Y_train=data.iloc[:1460-len(outlier_ids)]['SalePrice']
print('data after removing: ',data.shape)
print('training rows after removing outliers: ',Y_train.shape[0])
data.drop(columns=['Id','SalePrice'],inplace=True)
update_cols()


158 outliers found
[13, 47, 54, 59, 71, 73, 81, 94, 105, 113, 116, 123, 167, 172, 177, 186, 190, 197, 198, 200, 209, 225, 232, 250, 252, 272, 276, 279, 287, 290, 293, 298, 313, 314, 336, 340, 350, 352, 360, 361, 367, 370, 373, 385, 386, 401, 412, 427, 441, 452, 458, 467, 471, 472, 476, 488, 496, 504, 511, 518, 520, 524, 530, 551, 557, 561, 564, 584, 585, 589, 596, 602, 606, 619, 625, 626, 636, 648, 662, 674, 692, 707, 724, 736, 744, 756, 764, 765, 770, 786, 796, 799, 804, 809, 829, 831, 834, 849, 853, 855, 860, 876, 888, 889, 908, 913, 920, 935, 945, 982, 996, 1007, 1009, 1013, 1027, 1038, 1056, 1058, 1068, 1071, 1095, 1108, 1123, 1128, 1140, 1152, 1155, 1156, 1169, 1170, 1172, 1183, 1185, 1191, 1229, 1246, 1256, 1260, 1274, 1282, 1283, 1285, 1288, 1294, 1299, 1302, 1321, 1329, 1352, 1372, 1387, 1397, 1406, 1409, 1415, 1421, 1422, 1444]
data before removing:  (2919, 58)
data after removing:  (2761, 58)
training rows after removing outliers:  1302


In [982]:
data.iloc[:(1460-outliers_removed),:].to_csv('Xtrain.csv',index=False)
data.iloc[(1460-outliers_removed):,:].to_csv('Xtest.csv',index=False)
Y_train.to_csv('Ytrain.csv',index=False)

